<a href="https://colab.research.google.com/github/jmsmg/TIL/blob/main/Python/Concurrency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 병행성(Concurrency)

- 병행성, 흐름제어 설명
- 이터레이터(Iterator) : 반복 가능 객체
- 제너레이터(Generator) : 반복 가능한 객체를 생산해내는 역할 
- \_\_iter__, \_\_next__
- 클래스 기반 제너레이터 구현

In [ ]:
# Iterator, Generator
# 파이썬 반복 가능한 객체
# collections, text file, list, dict, set, tuple, unpacking, *args...

# 반복 가능한 이유 -> iter(x) 함수 호출
t = 'ABCD'

print(dir(t))

print('------')

# for c in t:
#   print(c)

print('------')

w = iter(t)

print(next(w))
print(next(w))

while True:
  try:
    print(next(w))
  except StopIteration:
    break

print('------')

# 반복형 확인

from collections import abc

# print(dir(t))
print(hasattr(t, '__iter__'))
print(isinstance(t, abc.Iterable))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']
------
------
A
B
C
D
------
True
True


In [ ]:
# next
class WordSplitter:
  def __init__(self, text):
    self._idx = 0
    self._text = text.split(' ')

  def __next__(self):
    print('Called __next__')
    try:
      word = self._text[self._idx]
    except IndexError:
      raise StopIteration('Stopped Iteration')
    self._idx += 1
    return word
  
  def __repr__(self):
    return 'WordSplit(%s)' % (self._text)

wi = WordSplitter('Do today what you could do tommorow')
print(next(wi))
print(next(wi))

# Generator 패턴
# 1. 지능형 리스트, 딕셔너리, 집합 -> 데이터 양 증가 후 메모리 사용량 증가 -> 제너레이터 사용 권장
# 2. 단위 실행 가능한 코루틴(coroutine)
# 3. 작은 메모리 조각 사용

class WordSplitGenerator:
  def __init__(self, text):
    self._text = text.split(' ')
  
  def __iter__(self):
    for word in self._text:
      yield word # 제너레이터 # yield로 상태 저장
    return

  def __repr__(self):
    return 'WordSplitGenerator(%s)' % (self._text)

wg = WordSplitGenerator('Do today what you could do tommorow')

wt = iter(wg)

Called __next__
Do
Called __next__
today


- 병행성(Concurrency) : 한 컴퓨터가 여러 일을 동시에 수행 (마지막 지점을 알아야함 = 클로저) -> 단일 프로그램안에서 여러 일을 쉽게 처리

- 병렬성(Parallelism) : 여러 컴퓨터가 여러 작업을 동시에 수행 -> 속도

In [ ]:
# Generator Ex1

def generator_ex1():
  print('Start')
  yield 'A Point'
  print('Continue')
  yield 'B Point'
  print('End')

temp = iter(generator_ex1())

# print(temp)
print(next(temp))

for v in generator_ex1():
  pass
  print(v)

# Generator Ex2
temp2 = [x * 3 for x in generator_ex1()]
temp3 = (x * 3 for x in generator_ex1())

print('----')
print(temp2)

for i in temp3:
  print(i)

Start
A Point
Start
A Point
Continue
B Point
End
Start
Continue
End
----
['A PointA PointA Point', 'B PointB PointB Point']
Start
A PointA PointA Point
Continue
B PointB PointB Point
End


In [ ]:
# Generator Ex3(중요 함수)
# count, takewhile, filterfalse, accumulate, chain, product, groupby...

import itertools

gen1 = itertools.count(1, 2.5)

print(next(gen1))
print(next(gen1))
# ... 무한

# 조건
gen2 = itertools.takewhile(lambda n : n < 50, gen1)

for v in gen2:
  print(v)

print('-----')

# 필터 반대
gen3 = itertools.filterfalse(lambda n : n < 3, [1, 2 ,3 ,4, 5])

for v in gen3:
  print(v)

print('-----')

# 누적 합계
gen4 = itertools.accumulate([x for x in range(1, 10)])

for v in gen4:
  print(v)

print('-----')

# 연결
gen5 = itertools.chain('ABCDE', range(1, 11, 2))

print(list(gen5))

# 연결2
gen6 = itertools.chain(enumerate('ABCDE'))
print(list(gen6))

1
3.5
6.0
8.5
11.0
13.5
16.0
18.5
21.0
23.5
26.0
28.5
31.0
33.5
36.0
38.5
41.0
43.5
46.0
48.5
-----
3
4
5
-----
1
3
6
10
15
21
28
36
45
-----
['A', 'B', 'C', 'D', 'E', 1, 3, 5, 7, 9]
[(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E')]


In [ ]:
# 개별 경우의 수
gen7 = itertools.product('ABCDE')

print(list(gen7))

gen8 = itertools.product('ABCDE', repeat=4)

print(list(gen8))

# 그룹화
gen9 = itertools.groupby('AAABBBCCCDDD')

# print(list(gen9))

for chr, group in gen9:
  print(chr, ' : ', list(group))

[('A',), ('B',), ('C',), ('D',), ('E',)]
[('A', 'A', 'A', 'A'), ('A', 'A', 'A', 'B'), ('A', 'A', 'A', 'C'), ('A', 'A', 'A', 'D'), ('A', 'A', 'A', 'E'), ('A', 'A', 'B', 'A'), ('A', 'A', 'B', 'B'), ('A', 'A', 'B', 'C'), ('A', 'A', 'B', 'D'), ('A', 'A', 'B', 'E'), ('A', 'A', 'C', 'A'), ('A', 'A', 'C', 'B'), ('A', 'A', 'C', 'C'), ('A', 'A', 'C', 'D'), ('A', 'A', 'C', 'E'), ('A', 'A', 'D', 'A'), ('A', 'A', 'D', 'B'), ('A', 'A', 'D', 'C'), ('A', 'A', 'D', 'D'), ('A', 'A', 'D', 'E'), ('A', 'A', 'E', 'A'), ('A', 'A', 'E', 'B'), ('A', 'A', 'E', 'C'), ('A', 'A', 'E', 'D'), ('A', 'A', 'E', 'E'), ('A', 'B', 'A', 'A'), ('A', 'B', 'A', 'B'), ('A', 'B', 'A', 'C'), ('A', 'B', 'A', 'D'), ('A', 'B', 'A', 'E'), ('A', 'B', 'B', 'A'), ('A', 'B', 'B', 'B'), ('A', 'B', 'B', 'C'), ('A', 'B', 'B', 'D'), ('A', 'B', 'B', 'E'), ('A', 'B', 'C', 'A'), ('A', 'B', 'C', 'B'), ('A', 'B', 'C', 'C'), ('A', 'B', 'C', 'D'), ('A', 'B', 'C', 'E'), ('A', 'B', 'D', 'A'), ('A', 'B', 'D', 'B'), ('A', 'B', 'D', 'C'), ('A', 'B', '

- 코루틴 : 단일(싱글) 스레드, 스택을 기반으로 작동하는 비동기 작업
  - 루틴 실행 중 중지 -> 동시성 프로그래밍
  - 쓰레드에 비해 오버헤드 감소

- 쓰레드
  - os에서 직접 관리
  - 시분할 비동기 작업 -> 멀티쓰레드
  - 싱글쓰레드 -> 멀티쓰레드 -> 코딩하기 복잡 -> 공유되는 자원 -> 교착 상태 발생 가능성, **컨텍스트 스위칭 비용 발생**, 자원 소비 가능성 증가

- Yield
  - 메인 <-> 서브
  - 코루틴 제어, 상태, 양방향 전송

- 서브루틴 : 메인루틴 호출 -> 서브루틴에서 수행(흐름제어)

In [10]:
# 코루틴 Ex1
def coroutine1():
  print('>>> coroutine started')
  i = yield
  print(f'>>> coroutine received : {i}')

# 제너레이터 선언
cr1 = coroutine1()

print(cr1, type(cr1))

# yield 지점까지 서브루틴 수행
next(cr1)

# 기본 전달값 None
# 값 전송

cr1.send(1) # 메인루틴과 서브루틴이 데이터 교환 가능

<generator object coroutine1 at 0x7f458aa3b850> <class 'generator'>
>>> coroutine started
>>> coroutine received : 1


StopIteration: ignored